In [76]:
import numpy as np
import math
import os
import gzip
import pickle
import random
import time
import copy

from sklearn.metrics import log_loss
from scipy.special import expit, softmax

from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical

In [77]:
NR_EPOCHS = 200
POP_SIZE = 30
ELITISM_NR = 10
HIGHER_BOUND = 1
LOWER_BOUND = -1
# 95% of values will be between LOWER_BOUND and HIGHER_BOUND
# if mean centered
SCALE = ((HIGHER_BOUND - LOWER_BOUND) / 2) / 2
INTERVALS_NR = (HIGHER_BOUND - LOWER_BOUND) * 10 ** 4
BITS_NR = math.ceil(np.log2(INTERVALS_NR))
MUTATION_PROB = 0.1
CROSSOVER_PROB = 0.6
BATCH_SIZE = 256
# 1 input, 1 hidden, 1 output = 3 layers
N_UNITS = [784, 16, 10]
N_WEIGHTS = len(N_UNITS) - 1
N_BIASES = N_WEIGHTS

In [78]:
# Activation functions
# TODO: See which is more efficient
def sigmoid(z):
    return np.divide(1, (1 + np.exp(-z)))


def expit_approx(x):
    return 1.0 / (1 + np.abs(x))


def softplus(x):
    return np.log(1 + np.exp(x))


# expit imported from scipy.special

In [79]:
def fitness_network(population, x, y):
    losses = []
    if not population:
        return []
    n_weights = len(population[0]) // 2
    for individual in population:
        weights = individual[:n_weights]
        biases = individual[n_weights:]
        y_pred = list()
        for start_idx in range(0, x.shape[0], BATCH_SIZE):
            x_batch = x[start_idx:start_idx + BATCH_SIZE]
            z = x_batch
            for i in range(n_weights - 1):
                z = np.dot(z, weights[i]) + biases[i]
                # expit may be better, although it's debatable.
                z = expit(z)
            z = np.dot(z, weights[n_weights - 1]) + biases[n_weights - 1]
            y_final = softmax(z)
            y_pred.append(y_final)
        y_pred = np.concatenate(y_pred)
        losses.append(1 / np.exp(log_loss(y, y_pred)))
    return losses


def test_network(individual, x, y):
    n_weights = len(individual) // 2
    weights = individual[:n_weights]
    biases = individual[n_weights:]
    y_pred = list()
    for start_idx in range(0, x.shape[0], BATCH_SIZE):
        x_batch = x[start_idx:start_idx + BATCH_SIZE]
        z = x_batch
        for i in range(n_weights - 1):
            z = np.dot(z, weights[i]) + biases[i]
            # expit may be better, although it's debatable.
            z = expit(z)
        z = np.dot(z, weights[n_weights - 1] + biases[n_weights] - 1)
        y_final = softmax(z)
        y_pred.append(y_final)
    y_pred = np.concatenate(y_pred)
    y_pred = np.apply_along_axis(np.argmax, 1, y_pred)
    return np.sum(y_pred == y) / y.size

In [80]:
def mutate(pop):
    new_pop = []
    for indiv in pop:
        new_indiv = []
        for layer in indiv:
            new_indiv.append(np.where(np.random.rand(*layer.shape) < MUTATION_PROB,
                                      layer + np.random.normal(loc=0,
                                                               scale=SCALE,
                                                               size=layer.shape),
                                      layer))
        new_pop.append(new_indiv)
    return new_pop


def crossover(pop, cross_percentages):
    def swap_weights(p, i1, i2):
        for i1_idx, i2_idx in zip(i1, i2):
            # choose a random layer (weights only)
            l = random.randint(0, N_WEIGHTS - 1)
            i = random.randint(0, p[i1_idx][l].shape[0] - 1)
            j = random.randint(0, p[i1_idx][l].shape[1] - 1)
            temp = p[i1_idx][l][i, j].copy()
            p[i1_idx][l][i, j] = p[i2_idx][l][i, j]
            p[i2_idx][l][i, j] = temp

    def swap_neurons(p, i1, i2):
        for i1_idx, i2_idx in zip(i1, i2):
            # choose a random layer (weights and biases)
            l = random.randint(0, N_WEIGHTS + N_BIASES - 1)
            i = random.randint(0, p[i1_idx][l].shape[0] - 1)
            temp = p[i1_idx][l][i].copy()
            p[i1_idx][l][i] = p[i2_idx][l][i]
            p[i2_idx][l][i] = temp

    def swap_layers(p, i1, i2):
        for i1_idx, i2_idx in zip(i1, i2):
            # choose a random layer (weights and biases)
            l = random.randint(0, N_WEIGHTS + N_BIASES - 1)
            temp = p[i1_idx][l].copy()
            p[i1_idx][l] = p[i2_idx][l]
            p[i2_idx][l] = temp

    def split_perc(indices, perc):
        # Turn percentages into values between 0 and 1
        splits = np.cumsum(perc)
        if splits[-1] != 1:
            raise ValueError("percents don't add up to 100")
        # Split doesn't need last percent, it will just take what is left
        splits = splits[:-1]
        # Turn values into indices
        splits *= len(indices)
        # Turn double indices into integers.
        # CAUTION: numpy rounds to closest EVEN number when a number is halfway
        # between two integers. So 0.5 will become 0 and 1.5 will become 2!
        # If you want to round up in all those cases, do
        # splits += 0.5 instead of round() before casting to int
        splits = splits.round().astype(np.int)
        splits = np.split(indices, splits)
        # Make arrays of even lengths
        for i in range(len(splits)):
            if len(splits[i]) % 2:
                splits[i] = np.append(splits[i],
                                      np.random.choice(splits[i],
                                                       size=(1,)))
        return splits

    # ACTUAL FUNCTION LOGIC STARTS HERE

    cross_indices = np.arange(POP_SIZE)[np.random.rand(POP_SIZE) < CROSSOVER_PROB]
    shuffled_indices = np.random.choice(cross_indices,
                                        size=cross_indices.size,
                                        replace=False)
    weights, neurons, layers = split_perc(shuffled_indices, cross_percentages)
    swap_weights(pop, *np.split(weights, 2))
    swap_neurons(pop, *np.split(neurons, 2))
    swap_layers(pop, *np.split(layers, 2))


def upgrade(population, cross_percentages=(.3, .3, .4)):
    new_population = mutate(population)
    # This function modifies the matrix in-place
    crossover(new_population, cross_percentages)
    return new_population

In [81]:
def selection(population, fitness_values, elitism=False):
    new_population = []
    # Compute cumulative distribution.
    total_fitness = sum(fitness_values)
    individual_probabilities = [fitness_val / total_fitness for fitness_val in fitness_values]
    cummulative_probabilities = np.cumsum(individual_probabilities)
    if not elitism:
        # Generate probabilities for new population.
        r = np.random.rand(POP_SIZE)
        # Get insertion points through a left bisect algorithm.
        selected = np.searchsorted(cummulative_probabilities, r)
        for idx in selected:
            new_population.append(population[idx])
    else:
        best_fitness_values = sorted(fitness_values, reverse=True)[:ELITISM_NR]
        chosen_elitism_values = [np.where(fitness_values == i)[0][0] for i in best_fitness_values]
        # Generate probabilities for new population.
        r = np.random.rand(POP_SIZE - ELITISM_NR)
        # Get insertion points through a left bisect algorithm.
        selected = np.searchsorted(cummulative_probabilities, r)
        for idx in selected:
            new_population.append(population[idx])
        for idx in chosen_elitism_values:
            new_population.append(population[idx])
    return new_population


def get_best_individual(population, fitness_values):
    local_best = np.argmax(fitness_values)
    best = fitness_values[local_best]
    best_individual = population[local_best]
    return best, best_individual

In [82]:
def generate_smart_population(x_train, y_train, load=False):
    if not load:
        input_layer = Input(shape=(784,))
        dense_1 = Dense(100, activation='sigmoid')(input_layer)
        dense_2 = Dense(10, activation='sigmoid')(dense_1)
        pred = Dense(10, activation='softmax')(dense_2)
        model = Model(inputs=input_layer, outputs=pred)
        model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['acc'])
        model.summary()
        model.fit(x_train, to_categorical(y_train, num_classes=10), batch_size=256, epochs=1)
        model.save('model.h5')
        loss, acc = model.evaluate(x_train, to_categorical(y_train))
    else:
        model = load_model('model.h5')
        loss, acc = model.evaluate(x_train, to_categorical(y_train))
    print(f'Accuracy from the initial model: {acc}')

    first_layer_weights = model.layers[1].get_weights()[0]
    first_layer_biases = model.layers[1].get_weights()[1]
    second_layer_weights = model.layers[2].get_weights()[0]
    second_layer_biases = model.layers[2].get_weights()[1]
    third_layer_weights = model.layers[3].get_weights()[0]
    third_layer_biases = model.layers[3].get_weights()[1]

    return [[np.copy(first_layer_weights), np.copy(second_layer_weights), np.copy(third_layer_weights),
             np.copy(first_layer_biases), np.copy(second_layer_biases), np.copy(third_layer_biases)]
            for _ in range(POP_SIZE)]


def generate_population(units=N_UNITS):
    return [[np.random.uniform(low=LOWER_BOUND,
                               high=HIGHER_BOUND,
                               size=(units[i], units[i+1])).astype('f')
            for i in range(len(units) - 1)]
            +
            [np.random.uniform(low=LOWER_BOUND,
                               high=HIGHER_BOUND,
                               size=(units[i+1],)).astype('f')
            for i in range(len(units) - 1)]
           for _ in range(POP_SIZE)]

In [83]:
def main(use_back_prop=True, load=True):
    start_time = time.time()
    with gzip.open('mnist.pkl.gz', 'rb') as f:
        train_set, _, test_set = pickle.load(f, encoding='latin1')
        x_train, y_train = train_set
        x_test, y_test = test_set
    if load:
        if os.path.exists('population.pkl'):
            with open('population.pkl', 'rb') as f:
                population = pickle.load(f)
        else:
            if not use_back_prop:
                population = generate_population()
            else:
                population = generate_smart_population(x_train, y_train, load=True)
    else:
        if not use_back_prop:
            population = generate_population()
        else:
            population = generate_smart_population(x_train, y_train, load=True)

    fitness_values = fitness_network(population, x_train, y_train)
    best, best_individual = get_best_individual(population, fitness_values)
    for i in range(NR_EPOCHS):
        if i % 10 == 0:
            with open('population.pkl', 'wb') as f:
                pickle.dump(population, f)
        print(f'Current epoch: {i}')
        population = selection(population, fitness_values, elitism=False)
        population = upgrade(population, cross_percentages=[.40, .55, .05])
        fitness_values = fitness_network(population, x_train, y_train)
        new_best, new_best_individual = get_best_individual(population, fitness_values)
        print('Current best:', best)
        print('New best:', new_best)
        if new_best > best:
            best = new_best
            best_individual = new_best_individual
            best_score = test_network(best_individual, x_train, y_train)
            print(f'The network achieved an accuracy of {best_score * 100} percent on training set!')
    best_score = test_network(best_individual, x_test, y_test)
    print(f'The network achieved an accuracy of {best_score * 100} percent on testing set!')
    print(f'Time taken: {time.time() - start_time} seconds!')

In [84]:
if __name__ == '__main__':
    main(use_back_prop=False, load=False)

Current epoch: 0
Current best: 0.07205270162192094
New best: 0.06730678191633749
Current epoch: 1
Current best: 0.07205270162192094
New best: 0.06621717138583273
Current epoch: 2
Current best: 0.07205270162192094
New best: 0.06351179306579609
Current epoch: 3
Current best: 0.07205270162192094
New best: 0.05859578786172433
Current epoch: 4
Current best: 0.07205270162192094
New best: 0.06381510724321734
Current epoch: 5
Current best: 0.07205270162192094
New best: 0.05760336254172088
Current epoch: 6
Current best: 0.07205270162192094
New best: 0.0619662614359498
Current epoch: 7
Current best: 0.07205270162192094
New best: 0.060222539917784416
Current epoch: 8
Current best: 0.07205270162192094
New best: 0.05986178982788527
Current epoch: 9
Current best: 0.07205270162192094
New best: 0.06071745266817565
Current epoch: 10
Current best: 0.07205270162192094
New best: 0.05437136009383444
Current epoch: 11
Current best: 0.07205270162192094
New best: 0.054991240775038266
Current epoch: 12
Current

KeyboardInterrupt: 